In [1]:
# ==============================================================================
# DTC Analytics: Final Data Preprocessing and Unification Script
# ==============================================================================
#
# Objective: This script transforms the 13 raw source CSV files into a single,
# clean, and non-redundant analytical dataset.
#
# Instructions:
# 1. Ensure all 13 source CSV files are in the same directory as this script.
# 2. Run the script to generate 'final_unified_dataset.csv'.
# ==============================================================================

# --- Stage 1: Setup and Helper Functions ---
print("--- Stage 1: Defining Helper Functions ---")

import pandas as pd
import numpy as np
import warnings

# Suppress potential warnings for cleaner output
warnings.filterwarnings('ignore')

def find_header_row(df):
    """Dynamically finds the most likely header row in a dataframe."""
    for i, row in df.iterrows():
        row_str = ' '.join(row.astype(str).str.upper().tolist())
        if ('YEAR' in row_str or 'CHANNEL' in row_str or 'KPI' in row_str or 'MONTH' in row_str or 'RETURNS' in row_str) and row.notna().sum() > 2:
            return i
    return -1

def clean_and_process_wide_format(file_path, metric_name, id_col_name='CHANNEL'):
    """Reads and processes wide-format files with months as columns."""
    try:
        df = pd.read_csv(file_path, header=None, dtype=str, encoding='latin-1')
    except FileNotFoundError:
        print(f"Info: File '{file_path}' not found. Skipping.")
        return pd.DataFrame()

    header_row_index = find_header_row(df)
    if header_row_index == -1:
        return pd.DataFrame()

    header = df.iloc[header_row_index].str.upper().str.strip()
    data = df.iloc[header_row_index + 1:].copy()
    data.columns = header

    id_col = next((col for col in [id_col_name, 'KPI'] if col in data.columns), data.columns[0])
    data = data.rename(columns={id_col: 'ID_COLUMN'})

    if 'YEAR' not in data.columns:
        return pd.DataFrame()
    if isinstance(data['YEAR'], pd.DataFrame):
        year_series = data['YEAR'].iloc[:, 0]
    else:
        year_series = data['YEAR']
    data['YEAR'] = pd.to_numeric(year_series, errors='coerce')
    data.dropna(subset=['YEAR'], inplace=True)
    data['YEAR'] = data['YEAR'].astype(int)
    
    month_map = {'JANUARY': 1, 'FEBRUARY': 2, 'MARCH': 3, 'APRIL': 4, 'MAY': 5, 'JUNE': 6,
                 'JULY': 7, 'AUGUST': 8, 'SEPTEMBER': 9, 'OCTOBER': 10, 'NOVEMBER': 11, 'DECEMBER': 12}
    month_cols = [col for col in data.columns if col in month_map]
    if not month_cols:
        return pd.DataFrame()

    melted = data.melt(id_vars=['ID_COLUMN', 'YEAR'], value_vars=month_cols, var_name='MONTH', value_name='value')
    melted['value'] = pd.to_numeric(melted['value'].astype(str).str.replace(r'[$,()]', '', regex=True), errors='coerce')
    melted.dropna(subset=['value'], inplace=True)
    melted['Date'] = pd.to_datetime(melted['YEAR'].astype(str) + '-' + melted['MONTH'].map(month_map).astype(str) + '-01')
    melted = melted.rename(columns={'ID_COLUMN': id_col_name})
    melted['metric_type'] = metric_name
    
    return melted[['Date', id_col_name, 'value', 'metric_type']]

print("Helper functions defined successfully.")


# --- Stage 2: Process Each Data Category ---
print("\n--- Stage 2: Processing all source files... ---")

file_processing_plan = [
    {'path': 'Media Spend.csv', 'metric': 'Spend', 'id_col': 'CHANNEL'},
    {'path': 'Media Spend by Channel.csv', 'metric': 'Spend', 'id_col': 'CHANNEL'},
    {'path': 'Technology Spend.csv', 'metric': 'Technology_Spend', 'id_col': 'Vendor'},
    {'path': 'Web Analytics.csv', 'metric': 'Sessions', 'id_col': 'CHANNEL'},
    {'path': 'Marketing Channel Breakdown.csv', 'metric': 'Clicks', 'id_col': 'CHANNEL'},
    {'path': 'Orders By Channel-New.csv', 'metric': 'Orders_New', 'id_col': 'CHANNEL'},
    {'path': 'Orders By Channel-Ext.csv', 'metric': 'Orders_Existing', 'id_col': 'CHANNEL'},
    {'path': 'Cust By Channel-New.csv', 'metric': 'Customers_New', 'id_col': 'CHANNEL'},
    {'path': 'Cust By Channel-Ext.csv', 'metric': 'Customers_Existing', 'id_col': 'CHANNEL'},
    {'path': 'TOPSHEET.csv', 'metric': 'Total_Revenue', 'id_col': 'KPI'}
]

all_dfs = []
for plan in file_processing_plan:
    df = clean_and_process_wide_format(plan['path'], plan['metric'], plan['id_col'])
    if not df.empty:
        df = df.rename(columns={plan['id_col']: 'ID'})
        all_dfs.append(df)

# Special handling for Returns.csv
try:
    returns_df = pd.read_csv('Returns.csv', header=0, dtype=str, encoding='latin-1')
    date_col = next((col for col in returns_df.columns if 'MONTH' in col.upper()), None)
    amount_col = next((col for col in returns_df.columns if 'RETURN' in col.upper()), None)
    if date_col and amount_col:
        returns_df['Date'] = pd.to_datetime(returns_df[date_col], errors='coerce')
        returns_df['value'] = returns_df[amount_col].str.replace(r'[($),]', '', regex=True).astype(float) * -1
        returns_monthly = returns_df[['Date', 'value']].dropna()
        returns_monthly['metric_type'] = 'Total_Returns'
        returns_monthly['ID'] = 'Business' # Assign a generic ID for business-level metrics
        all_dfs.append(returns_monthly)
except (FileNotFoundError, KeyError) as e:
    print(f"Info: Could not process 'Returns.csv'. Error: {e}. Skipping.")

print("Individual file processing complete.")


# --- Stage 3: Aggregation and Final Unification ---
print("\n--- Stage 3: Aggregating and unifying all data... ---")
from functools import reduce

full_df = pd.concat(all_dfs)

# Filter out summary rows and standardize channel names
full_df = full_df[~full_df['ID'].str.contains('TOTAL', na=False)]
full_df['ID'] = full_df['ID'].str.replace(' Media', '', regex=False).str.strip()

# Separate business-level metrics from channel-level metrics
business_metrics = ['Total_Revenue', 'Total_Returns', 'Technology_Spend']
business_df = full_df[full_df['metric_type'].isin(business_metrics)]
channel_df = full_df[~full_df['metric_type'].isin(business_metrics)]

# Pivot the channel data to create the non-redundant structure
channel_pivot = channel_df.pivot_table(
    index=['Date', 'ID'], 
    columns='metric_type', 
    values='value', 
    aggfunc='sum'
).reset_index().rename(columns={'ID': 'CHANNEL'})

# Aggregate business data to a single value per date
business_agg = business_df.groupby(['Date', 'metric_type'])['value'].sum().unstack().reset_index()

# Merge the two dataframes together
final_df = pd.merge(channel_pivot, business_agg, on='Date', how='left')

print("Unification complete.")


# --- Stage 4: Final Cleanup and Saving ---
print("\n--- Stage 4: Performing final cleanup and saving... ---")

# Add YEAR and MONTH columns
final_df['YEAR'] = final_df['Date'].dt.year
final_df['MONTH'] = final_df['Date'].dt.month_name().str.upper()

# Fill any remaining NaN values with 0
numeric_cols = final_df.select_dtypes(include=np.number).columns
final_df[numeric_cols] = final_df[numeric_cols].fillna(0)

# Define final column order for clarity
final_cols = ['YEAR', 'MONTH', 'CHANNEL', 'Sessions', 'Clicks', 'Spend', 
              'Total_Returns', 'Technology_Spend', 'Total_Revenue', 
              'Orders_Existing', 'Customers_Existing', 'Orders_New', 'Customers_New']
existing_final_cols = [col for col in final_cols if col in final_df.columns]
final_df = final_df[existing_final_cols]

# Display final result sample
print("\n--- Final Preprocessed Data (First 5 Rows) --- ")
print(final_df.head().to_string())

# Save the final dataframe to a CSV file
output_filename = 'final_unified_dataset.csv'
final_df.to_csv(output_filename, index=False)

print(f"\nSuccessfully saved the final, non-redundant data to '{output_filename}'")



--- Stage 1: Defining Helper Functions ---
Helper functions defined successfully.

--- Stage 2: Processing all source files... ---
Individual file processing complete.

--- Stage 3: Aggregating and unifying all data... ---
Unification complete.

--- Stage 4: Performing final cleanup and saving... ---

--- Final Preprocessed Data (First 5 Rows) --- 
metric_type  YEAR    MONTH         CHANNEL  Spend  Total_Returns  Total_Revenue  Orders_Existing  Customers_Existing  Orders_New  Customers_New
0            2024  JANUARY       Affiliate    0.0       -61116.9     1397391.57            116.0                 0.0         0.0            0.0
1            2024  JANUARY          Direct    0.0       -61116.9     1397391.57            793.0                 0.0       800.0            0.0
2            2024  JANUARY         Display    0.0       -61116.9     1397391.57              0.0                58.0         0.0           24.0
3            2024  JANUARY           Email    0.0       -61116.9     1397